In [ ]:
import pandas as pd
import spacy
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV
from surprise.dump import dump, load

# Cargar datos
ruta_archivo = r'.\merged.csv'
resenas = pd.read_csv(ruta_archivo)
resenas = resenas[['name', 'category', 'rating', 'user_id', 'business_id']]

# Filtrar categorías
nlp = spacy.load("es_core_news_md")
palabras_clave = ['restaurante', 'comida', 'cocina']
categorias_relacionadas = []
for categoria in resenas['category'].astype(str).unique().tolist():
    similitud_maxima = max(nlp(categoria).similarity(nlp(palabra)) for palabra in palabras_clave)
    if similitud_maxima > 0.7:
        categorias_relacionadas.append(categoria)
resenas = resenas[resenas['category'].isin(categorias_relacionadas)]

# Exportar conjunto de datos
resenas.to_csv('datos_usuario_item.csv', index=False)

# Convertir tipos de datos
resenas['user_id'] = resenas['user_id'].astype(str)
resenas['business_id'] = resenas['business_id'].astype(str)

# Lector
lector = Reader(rating_scale=(resenas["rating"].min(), resenas["rating"].max()))
conjunto_datos = Dataset.load_from_df(resenas[['user_id', 'business_id', 'rating']], lector)

# GridSearchCV
parametros = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, parametros, measures=['fcp', 'rmse'], cv=3, refit=True)
gs.fit(conjunto_datos)

# Mejor modelo
mejor_modelo = gs.best_estimator['fcp']

# Guardar el modelo
modelo_guardado = "modelo_usuario_item.pkl"
dump(modelo_guardado, algo=mejor_modelo)

# Función de recomendación
def recomendacion_restaurante(id_restaurante, top_n=5):
    # Cargar el modelo entrenado
    modelo_cargado = load(modelo_guardado)

    # Crear lista de recomendaciones
    recomendaciones = []

    # Hacer predicciones para cada ítem no visto
    for id_item in resenas[~resenas['business_id'].isin([id_restaurante])]['business_id'].unique():
        pred = modelo_cargado.predict(id_restaurante, id_item)
        recomendaciones.append((id_item, pred.est))

    # Ordenar las recomendaciones por calificación estimada
    recomendaciones.sort(key=lambda x: x[1], reverse=True)

    # Devolver los N mejores elementos recomendados
    return [item[0] for item in recomendaciones[:top_n]]

# Ejemplo de uso
recomendaciones = recomendacion_restaurante("business_id")
print(recomendaciones)